In [8]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
sc.stop()

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[*]", "Joining DStream with RDD")
ssc = StreamingContext(sc, 10)
ssc.checkpoint("/Users/pravinkumar/Documents/Spark/Results/checkPointForStreaming")

In [9]:
# Create a DStream that will connect to hostname:port, like localhost:9999
socket = ssc.socketTextStream("localhost", 9999)

In [14]:
def getSparkSessionInstance(sparkConf):
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession\
            .builder\
            .config(conf=sparkConf)\
            .getOrCreate()
    return globals()['sparkSessionSingletonInstance']

# Convert RDDs of the words DStream to DataFrame and run SQL query
def process(time, rdd):
    print("========= %s =========" % str(time))

    
    # Get the singleton instance of SparkSession
    spark = getSparkSessionInstance(rdd.context.getConf())

    # Convert RDD[String] to RDD[Row] to DataFrame
    rowRdd = rdd.map(lambda rec: Row(color = rec[0], number = rec[1]))
    deckRDD = spark.sparkContext.textFile("/Users/pravinkumar/Documents/Spark/testData/cards/deckofcards.txt").map(lambda rec: rec.split('|')).map(lambda rec: Row(color = rec[0], number = rec[2]))

    deckJoin = rowRdd.join(deckRDD)

    for i in deckJoin.take(10): print(i)
    

In [15]:
# formatting deckDstream and deckRDD for the join
deckDstream = socket.map(lambda rec: rec.split("|"))
#deckRDD = sc.textFile("/Users/pravinkumar/Documents/Spark/testData/cards/deckofcards.txt").map(lambda rec: rec.split('|')).map(lambda rec: Row(color = rec[0], number = rec[2]))


In [16]:
# Joining each batch of DStream with deckRDD 

deckDstream.foreachRDD(process)

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTerminationOrTimeout(300)  # Waiting for 5 min before terminating the computation.
ssc.stop()              # stoping the computation if timeout happens.

========= 2017-07-09 23:17:20 =========
========= 2017-07-09 23:17:30 =========
========= 2017-07-09 23:17:40 =========
========= 2017-07-09 23:17:50 =========
========= 2017-07-09 23:18:00 =========
========= 2017-07-09 23:18:10 =========
========= 2017-07-09 23:18:20 =========
========= 2017-07-09 23:18:30 =========
========= 2017-07-09 23:18:40 =========
========= 2017-07-09 23:18:50 =========
========= 2017-07-09 23:19:00 =========
========= 2017-07-09 23:19:10 =========
========= 2017-07-09 23:19:20 =========
========= 2017-07-09 23:19:30 =========
